In [2]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd

In [3]:
# accessing google APIs
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
# credentials to access files and responses
creds = ServiceAccountCredentials.from_json_keyfile_name("secrets.json", scope)
# connecting to a client
client = gspread.authorize(creds)


In [6]:
for ind, sheet in enumerate(client.openall()):
    print(sheet.title, sheet.id)

Customer Feedback (Responses) 1Z8nQZ97DXhbOdoSuvY2mL6yT6UayQQ44HXeYcEVLfLY
T-Shirt Sign Up (Responses) 1ppRxMKaEjhCPWsNBkH52MGAZ1Y5x0O4JjGz_Hl26VeA
RSVP (Responses) 1WN5Zw8NyZK3CpQjSp6dLHvzDWRBmZC96w9DHOOT4SI4
Party Invite (Responses) 1TD7GVp9QBp3V69AbgngS1qB6UUVN7ehCOV7V0A1tGCY
Event Registration (Responses) 1t-nj4nQdP3d5lSuH8BG43Ifg7_-Gv0jkApK6BSsDLBQ


In [172]:
new_rows = get_new_rows("Customer Feedback (Responses)")

In [170]:
new_rows

,Timestamp,Name,Email,Rating,Feedback,Suggestion for Improvement
5,7/4/2021 20:05:00,Kyle,kyle@gmail.com,3,ok service,Fun


In [159]:
def get_new_rows(sheet_name):
    sh = client.open(sheet_name)
    worksheet = sh.get_worksheet(0)
    df = pd.DataFrame(worksheet.get_all_records())
    worksheet_list = [ws.title for ws in sh.worksheets()]
    if "Marker" not in worksheet_list:
        sh.add_worksheet(title="Marker", rows="4000", cols="1000")
        marker_sheet = sh.worksheet("Marker")
        marker_sheet.update('A1', 'Processed')
        return df
    else:
        marker=sh.worksheet("Marker")
        marker_df = pd.DataFrame(marker.get_all_records())
        concatenated = pd.concat([df,marker_df],axis=1)
        df = concatenated[concatenated["Processed"]!="TRUE"]
        return df.drop("Processed",axis=1)
    

In [173]:
mark_unmarked_rows("Customer Feedback (Responses)")

In [167]:
def mark_unmarked_rows(sheet_name):
    sh = client.open(sheet_name)
    worksheet = sh.get_worksheet(0)
    df = pd.DataFrame(worksheet.get_all_records())
    marker_sheet = sh.worksheet("Marker")
    max_rows = df.shape[0]
    for i in range(2,max_rows+2):
        marker_sheet.update("A{}".format(i),"TRUE")
    

TypeError: can only concatenate str (not "int") to str